In [1]:
from openmm.app import *
from openmm import *
from openmm import unit as u
import sys
import mdtraj as md
import numpy as np
from tqdm import tqdm

In [2]:
# Nombre del archivo PDB del péptido
pdb_filename = "pdbs/V5.pdb"

In [3]:
# Cargar la estructura del péptido desde el archivo PDB
pdb = PDBFile(pdb_filename)

In [4]:
# Definir la fuerza de campo (AMBER ff14SB es común para péptidos)
forcefield = ForceField("amber14-all.xml", "amber14/tip3p.xml")

In [5]:
# Crear el sistema en vacío (sin agua ni iones)
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=NoCutoff,  # Sin tratamiento de fronteras periódicas
    constraints=HBonds         # Restringe enlaces de hidrógeno para un paso de integración más largo
)

In [6]:
# Configurar el integrador (Langevin dynamics)
temperature = 300 * u.kelvin
friction = 1 / u.picosecond
timestep = 2 * u.femtoseconds
integrator = LangevinIntegrator(temperature, friction, timestep)

In [7]:
# Crear una plataforma para ejecutar la simulación (e.g., CUDA si tienes GPU)
platform = Platform.getPlatformByName("CUDA")  # Usa "CPU" si no tienes GPU

In [8]:
# Configurar la simulación
simulation = Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [9]:
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy()

In [10]:
print('Energy before minimization:', energy)

Energy before minimization: 50835052.372887135 kJ/mol


In [11]:
# Minimización de energía
print("Minimizando energía...")
simulation.minimizeEnergy()

Minimizando energía...


In [12]:
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy()
print('Energy after minimization:', energy)

Energy after minimization: 141.82317173480988 kJ/mol


In [13]:
md_topology = md.Topology.from_openmm(simulation.topology)

In [14]:
steps_segment = 1000
n_segments = 1000

In [15]:
def es_el_mismo(angs1, angs2):
    if np.max(np.abs(angs1-angs2))<0.1:
        return True
    else:
        return False

In [21]:
traj_inh_db = []
unique_db = []

In [22]:
for ii in tqdm(range(n_segments)):
    simulation.step(steps_segment)
    state = simulation.context.getState(getEnergy=True)
    energy = state.getPotentialEnergy()
    simulation.minimizeEnergy()
    min_state = simulation.context.getState(getEnergy=True, getPositions=True)
    min_energy = min_state.getPotentialEnergy()
    min_positions = min_state.getPositions(asNumpy=True)
    traj = md.Trajectory(min_positions / u.nanometer, md_topology)
    phis = md.compute_phi(traj)[1]
    psis = md.compute_psi(traj)[1]
    dihed_angs=np.concatenate((phis[0],psis[0]))
    visitado = False
    for unique_index in range(len(unique_db)):
        aux = es_el_mismo(dihed_angs, unique_db[unique_index])
        if aux == True:
            visitado = True
            traj_inh_db.append(unique_index)
            break
    if visitado == False:
        traj_inh_db.append(len(unique_db))
        unique_db.append(dihed_angs)
    #print(ii, energy, min_energy, phis, psis)
    #print(dihed_angs)

100%|██████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:45<00:00, 21.95it/s]


In [25]:
len(unique_db)

23

In [26]:
traj_inh_db

[0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 3,
 4,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 3,
 1,
 1,
 2,
 5,
 1,
 1,
 1,
 1,
 6,
 1,
 1,
 2,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 1,
 2,
 7,
 1,
 1,
 1,
 8,
 1,
 1,
 1,
 6,
 2,
 1,
 2,
 9,
 1,
 0,
 1,
 10,
 1,
 1,
 1,
 1,
 4,
 1,
 8,
 11,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 5,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 12,
 6,
 2,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 5,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 6,
 2,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 11,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 0,
 1,
 1,
 2,
 6,
 2,
 1,
 3,
 2,
 5,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 13,
 1,
 5,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 1,
 2,
 3,
 1,
 1,
 1,
 0,
 6,
 9,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 5,
 4,
 1,
 1,
 1,
 3,
 5,
 8,
 1,
 1,
 2,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 7,
 13,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 13,
 7,
 1,
 